<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Soft_Actor_Critic_(SAC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gymnasium stable_baselines3

In [ ]:
import gymnasium as gym
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv

# Create the environment using Gymnasium
env = gym.make('Pendulum-v1')

# Wrap the environment with DummyVecEnv for compatibility
env = DummyVecEnv([lambda: env])

# Initialize the SAC model
model = SAC('MlpPolicy', env, verbose=1)

# Train the model
model.learn(total_timesteps=10000)

# Evaluate the trained model
obs = env.reset()
for _ in range(1000):
    action, _states = model.predict(obs)
    result = env.step(action)  # Adjusted to handle latest Gymnasium API
    if len(result) == 5:
        obs, reward, done, trunc, info = result
        done = done or trunc
    else:
        obs, reward, done, info = result
    env.render()
    # Handle the done flag appropriately
    if done:
        obs = env.reset()
env.close()